In [2]:
import os
import glob
from tqdm import tqdm

base_path = os.getcwd()
tibetan_lines = []
english_lines = []

# Get all toh directories
toh_path = os.path.join(base_path, 'data-tm-alignments/aligned/toh*')
toh_dirs = glob.glob(toh_path)

In [3]:
pairs = []

for toh_dir in tqdm(toh_dirs, desc="Processing directories"):
    # Look for *-bo.txt and *-en.txt files
    bo_files = glob.glob(os.path.join(toh_dir, '*-bo.txt'))

    for bo_file in tqdm(bo_files, desc=f"Processing files in {os.path.basename(toh_dir)}", leave=False):
        en_file = bo_file.replace('-bo.txt', '-en.txt')

        if os.path.exists(en_file):
            with open(bo_file, 'r', encoding='utf-8') as f:
                bo_lines = [line for line in f]

            with open(en_file, 'r', encoding='utf-8') as f:
                en_lines = [line for line in f]

        pairs.extend([{'bo': bo_elt.strip(), 'en': en_elt.strip(), 'source_file': toh_dir[-6:]} for bo_elt, en_elt in zip(bo_lines, en_lines)])

Processing directories: 100%|██████████| 152/152 [00:00<00:00, 164.23it/s]


In [4]:
pairs

[{'bo': '༄༅༅། །རྒྱ་གར་སྐད་དུ། ཨཱརྱ་སུ་བརྞྞ་བཱ་ལུ་ཀོ་པ་མ་ནཱ་མ་མ་ཧཱ་ཡཱ་ན་སཱུ་ཏྲ། བོད་སྐད་དུ།',
  'en': '',
  'source_file': 'toh126'},
 {'bo': 'འཕགས་པ་གསེར་གྱི་བྱེ་མ་ལྟ་བུ་ཞེས་བྱ་བ་ཐེག་པ་ཆེན་པོའི་མདོ།',
  'en': 'The Noble Mahāyāna Sūtra Like Gold Dust',
  'source_file': 'toh126'},
 {'bo': 'སངས་རྒྱས་དང་། བྱང་ཆུབ་སེམས་དཔའ་ཐམས་ཅད་ལ་ཕྱག་འཚལ་ལོ། །',
  'en': 'Homage to all buddhas and bodhisattvas.',
  'source_file': 'toh126'},
 {'bo': 'འདི་སྐད་བདག་གིས་ཐོས་པ་དུས་གཅིག་ན།',
  'en': 'Thus did I hear at one time.',
  'source_file': 'toh126'},
 {'bo': 'བཅོམ་ལྡན་འདས་མཉན་དུ་ཡོད་པ་ན་རྒྱལ་བུ་རྒྱལ་བྱེད་ཀྱི་ཚལ་མགོན་མེད་ཟས་སྦྱིན་གྱི་ཀུན་དགའ་ར་བ་ན་དགེ་སློང་སྟོང་ཉིས་བརྒྱ་ལྔ་བཅུའི་དགེ་སློང་གི་དགེ་འདུན་ཆེན་པོ་དང་། བྱང་ཆུབ་སེམས་དཔའ་སྟོང་ཕྲག་མང་པོ་དག་དང་ཐབས་ཅིག་ཏུ་བཞུགས་ཏེ།',
  'en': 'The Blessed One was staying at Prince Jeta’s Grove, Anāthapiṇḍada’s park in Śrāvastī, along with a great saṅgha of 1,250 monks and many thousands of bodhisattvas.',
  'source_file': 'toh126'},
 {'bo': 'དེ་ནས་ཚེ་དང་ལྡན་པ་ཀུན་དགའ་བོ

In [5]:
import pandas as pd

df = pd.DataFrame(pairs)
df.head()

,bo,en,source_file
0,༄༅༅། །རྒྱ་གར་སྐད་དུ། ཨཱརྱ་སུ་བརྞྞ་བཱ་ལུ་ཀོ་པ་མ...,,toh126
1,འཕགས་པ་གསེར་གྱི་བྱེ་མ་ལྟ་བུ་ཞེས་བྱ་བ་ཐེག་པ་ཆེན...,The Noble Mahāyāna Sūtra Like Gold Dust,toh126
2,སངས་རྒྱས་དང་། བྱང་ཆུབ་སེམས་དཔའ་ཐམས་ཅད་ལ་ཕྱག་འཚ...,Homage to all buddhas and bodhisattvas.,toh126
3,འདི་སྐད་བདག་གིས་ཐོས་པ་དུས་གཅིག་ན།,Thus did I hear at one time.,toh126
4,བཅོམ་ལྡན་འདས་མཉན་དུ་ཡོད་པ་ན་རྒྱལ་བུ་རྒྱལ་བྱེད་...,The Blessed One was staying at Prince Jeta’s G...,toh126


In [6]:
problems = []
for idx, row in enumerate(df.iterrows()):
    if 'སངས་རྒྱས' in row[1]['bo'] and "buddha" not in row[1]['en'].lower():
        problem = f'Row {row[0]}: སངས་རྒྱས in Tibetan but Buddha not in English'
        print(problem)
        problems.append((row[0], problem))
    elif 'སངས་རྒྱས' not in row[1]['bo'] and "buddha" in row[1]['en'].lower():
        problem = f'Row {row[0]}: སངས་རྒྱས not in Tibetan but Buddha in English'
        print(problem)
        problems.append((row[0], problem))

Row 54: སངས་རྒྱས in Tibetan but Buddha not in English
Row 57: སངས་རྒྱས in Tibetan but Buddha not in English
Row 72: སངས་རྒྱས not in Tibetan but Buddha in English
Row 73: སངས་རྒྱས in Tibetan but Buddha not in English
Row 75: སངས་རྒྱས not in Tibetan but Buddha in English
Row 85: སངས་རྒྱས not in Tibetan but Buddha in English
Row 222: སངས་རྒྱས in Tibetan but Buddha not in English
Row 223: སངས་རྒྱས not in Tibetan but Buddha in English
Row 225: སངས་རྒྱས not in Tibetan but Buddha in English
Row 242: སངས་རྒྱས not in Tibetan but Buddha in English
Row 254: སངས་རྒྱས not in Tibetan but Buddha in English
Row 277: སངས་རྒྱས not in Tibetan but Buddha in English
Row 292: སངས་རྒྱས not in Tibetan but Buddha in English
Row 308: སངས་རྒྱས not in Tibetan but Buddha in English
Row 310: སངས་རྒྱས not in Tibetan but Buddha in English
Row 312: སངས་རྒྱས not in Tibetan but Buddha in English
Row 314: སངས་རྒྱས not in Tibetan but Buddha in English
Row 318: སངས་རྒྱས in Tibetan but Buddha not in English
Row 337: སངས་རྒྱ

Original number of detected problems is 5584

After alteration: 2070

In [7]:
len(problems)

2070

bo: 'སློབ་དཔོན་གྱིས་བསྒོ་བ་མ་མཉན་པ་དང་། ཉན་ཐོས་ཀྱི་ཐེག་པ་ལ་ཡང་དག་པར་ཞུགས་པའམ། རང་སངས་རྒྱས་ཀྱི་ཐེག་པ་ལ་ཡང་དག་པར་ཞུགས་པའམ། ཐེག་པ་ཆེན་པོ་ལ་ཡང་དག་པར་ཞུགས་པའམ།'

en: 'I have disobeyed the upādhyāyas and the ācāryas.'

Monlam: Not listening to the teacher's instructions, or truly engaging in the Hearer's Vehicle, or truly engaging in the Solitary Buddha's Vehicle, or truly engaging in the Great Vehicle.


In [14]:
df.iloc[222]['en']

'I have disobeyed the upādhyāyas and the ācāryas.'

bo: 'དེ་ལས་གཞན་པའི་སེམས་ཅན་རྣམས་ལ་མི་སྙན་པར་བགྱིས་པ་དང་། སྤྱོས་པ་དང་། བརྡེགས་པ་དང་། དེ་དག་ལ་ཐ་བ་དང་། ཞེ་སྡང་བསྐྱེད་པ་དང་།'

en: 'I have been unpleasant to those who have entered the Śrāvakayāna, entered the Pratyekabuddhayāna, entered the Mahāyāna, and every other being—I have reviled and beaten them with a mind that is obstinate, angry, envious, and miserly.'

Monlam: And to cause unpleasantness, scolding, beating, and generating contempt and hatred towards other sentient beings.

In [15]:
df.iloc[223]['en']

'I have been unpleasant to those who have entered the Śrāvakayāna, entered the Pratyekabuddhayāna, entered the Mahāyāna, and every other being—I have reviled and beaten them with a mind that is obstinate, angry, envious, and miserly.'

In [17]:
len(df)

135943

In [25]:
import numpy as np

na_df = df.copy()

na_df['bo'].replace('', np.nan)
na_df['en'].replace('', np.nan, inplace=True)

/tmp/ipykernel_28736/4165528799.py:6: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  na_df['en'].replace('', np.nan, inplace=True)


In [26]:
na_df.dropna(inplace=True)
len(na_df)

135620

In [27]:
from datasets import Dataset

ds = Dataset.from_pandas(na_df)
ds[0]

{'bo': 'འཕགས་པ་གསེར་གྱི་བྱེ་མ་ལྟ་བུ་ཞེས་བྱ་བ་ཐེག་པ་ཆེན་པོའི་མདོ།',
 'en': 'The Noble Mahāyāna Sūtra Like Gold Dust',
 'source_file': 'toh126',
 '__index_level_0__': 1}

In [31]:
ds = ds.remove_columns(['__index_level_0__'])
ds[0]

{'bo': 'འཕགས་པ་གསེར་གྱི་བྱེ་མ་ལྟ་བུ་ཞེས་བྱ་བ་ཐེག་པ་ཆེན་པོའི་མདོ།',
 'en': 'The Noble Mahāyāna Sūtra Like Gold Dust',
 'source_file': 'toh126'}

In [32]:
ds.push_to_hub('billingsmoore/84000-bo-en')

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/136 [00:00<?, ?ba/s]

CommitInfo(commit_url='https://huggingface.co/datasets/billingsmoore/84000-bo-en/commit/bff80b1f6f8b223ea8c26afa048ad3bd0e2d7480', commit_message='Upload dataset', commit_description='', oid='bff80b1f6f8b223ea8c26afa048ad3bd0e2d7480', pr_url=None, repo_url=RepoUrl('https://huggingface.co/datasets/billingsmoore/84000-bo-en', endpoint='https://huggingface.co', repo_type='dataset', repo_id='billingsmoore/84000-bo-en'), pr_revision=None, pr_num=None)